In [1]:
try:
    from tfx import v1 as tfx
    
    if tfx.__version__ != "1.4.0":
        raise ModuleNotFoundError
except ModuleNotFoundError:
    !pip install tfx==1.4.0

In [2]:
import tensorflow as tf

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

print('TFX version: {}'.format(tfx.__version__))
print('TensorFlow version: {}'.format(tf.__version__))

TFX version: 1.4.0
TensorFlow version: 2.6.2


In [3]:
import os
from absl import logging

logging.set_verbosity(logging.INFO)

PIPELINE_NAME = "iris"
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

DATA_ROOT = os.path.join("data")
os.makedirs(DATA_ROOT, exist_ok=True)

# Retrieve data

In [4]:
import urllib.request

_data_url = 'https://datahub.io/machine-learning/iris/r/iris.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")

urllib.request.urlretrieve(_data_url, _data_filepath)

!head {_data_filepath}

# Create TFX pipeline and run it

In [5]:
_module_file = 'iris_utils.py'

In [6]:
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     module_file: str, serving_model_dir: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
    # Brings data into the pipeline or otherwise joins/converts training data.
    example_gen = tfx.components.CsvExampleGen(
        input_base=data_root)

    # Computes statistics over data for visualization and example validation.
    statistics_gen = tfx.components.StatisticsGen(
        examples=example_gen.outputs['examples'])

    # Generate a schema based on your data statistics.
    # A schema defines the expected bounds, types, and properties of the features in your dataset.
    schema_gen = tfx.components.SchemaGen(
        statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)

    # Performs anomaly detection based on statistics and data schema.
    example_validator = tfx.components.ExampleValidator(
        statistics=statistics_gen.outputs['statistics'],
        schema=schema_gen.outputs['schema'])

    # Transforms input data using preprocessing_fn in the 'module_file'.
    transform = tfx.components.Transform(
        examples=example_gen.outputs['examples'],
        schema=schema_gen.outputs['schema'],
        materialize=False,
        module_file=module_file)
    
    tuner = tfx.components.Tuner(
        module_file=module_file,
        examples=example_gen.outputs['examples'],
        schema=schema_gen.outputs['schema'],
        transform_graph=transform.outputs['transform_graph'],
        train_args=tfx.proto.TrainArgs(num_steps=20),
        eval_args=tfx.proto.EvalArgs(num_steps=5))


    # Uses user-provided Python function that trains a model.
    trainer = tfx.components.Trainer(
        module_file=module_file,
        examples=example_gen.outputs['examples'],
        schema=schema_gen.outputs['schema'],
        transform_graph=transform.outputs['transform_graph'],
        hyperparameters=tuner.outputs['best_hyperparameters'],
        train_args=tfx.proto.TrainArgs(num_steps=2000),
        eval_args=tfx.proto.EvalArgs(num_steps=5))

    # Pushes the model to a filesystem destination.
    pusher = tfx.components.Pusher(
        model=trainer.outputs['model'],
        push_destination=tfx.proto.PushDestination(filesystem=tfx.proto.PushDestination
                                                   .Filesystem(base_directory=serving_model_dir))
    )

    components = [
        example_gen,
        statistics_gen,
        schema_gen,
        example_validator,
        transform,
        tuner,
        trainer,
        pusher,
    ]

    return tfx.dsl.Pipeline(
        pipeline_name=pipeline_name,
        pipeline_root=pipeline_root,
        metadata_connection_config=tfx.orchestration.metadata.sqlite_metadata_connection_config(metadata_path),
        components=components)

In [7]:
pipeline = _create_pipeline(
    pipeline_name=PIPELINE_NAME,
    pipeline_root=PIPELINE_ROOT,
    data_root=DATA_ROOT,
    module_file=_module_file,
    serving_model_dir=SERVING_MODEL_DIR,
    metadata_path=METADATA_PATH
)

tfx.orchestration.LocalDagRunner().run(pipeline)

Trial 6 Complete [00h 00m 01s]
val_accuracy: 0.8999999761581421

Best val_accuracy So Far: 0.9200000166893005
Total elapsed time: 00h 00m 07s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit
INFO:absl:Finished tuning... Tuner ID: tuner0
INFO:absl:Best HyperParameters: {'space': [{'class_name': 'Int', 'config': {'name': 'dnn_hidden_layer_0', 'default': None, 'conditions': [], 'min_value': 100, 'max_value': 150, 'step': 1, 'sampling': None}}, {'class_name': 'Int', 'config': {'name': 'dnn_hidden_layer_1', 'default': None, 'conditions': [], 'min_value': 50, 'max_value': 70, 'step': 1, 'sampling': None}}], 'values': {'dnn_hidden_layer_0': 135, 'dnn_hidden_layer_1': 63}}
INFO:absl:Best Hyperparameters are written to pipelines/iris/Tuner/best_hyperparameters/14/best_hyperparameters.txt.
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 14 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'best_hyperparameters': [Artifact(artifact: uri: "pipelines/iris/Tuner/best_hyperparameters/14"
custom_properties {
  key: "name"
  value {
    string_value: "iris:20

INFO:absl:Train on the 'train' split when train_args.splits is not set.
INFO:absl:Evaluate on the 'eval' split when eval_args.splits is not set.
INFO:absl:udf_utils.get_fn {'module_path': 'iris_utils@pipelines/iris/_wheels/tfx_user_code_Trainer-0.0+cdd0b8bea05713bd18c34d96752b4a5d9ef743c4bb5d652b4b7b2982c29698a3-py3-none-any.whl', 'train_args': '{\n  "num_steps": 2000\n}', 'eval_args': '{\n  "num_steps": 5\n}', 'custom_config': 'null'} 'run_fn'
INFO:absl:Installing 'pipelines/iris/_wheels/tfx_user_code_Trainer-0.0+cdd0b8bea05713bd18c34d96752b4a5d9ef743c4bb5d652b4b7b2982c29698a3-py3-none-any.whl' to a temporary directory.
INFO:absl:Executing: ['/home/ekrem/.pyenv/versions/3.8.7/envs/tfx/bin/python3.8', '-m', 'pip', 'install', '--target', '/tmp/tmp02mljda1', 'pipelines/iris/_wheels/tfx_user_code_Trainer-0.0+cdd0b8bea05713bd18c34d96752b4a5d9ef743c4bb5d652b4b7b2982c29698a3-py3-none-any.whl']


Results summary
Results in pipelines/iris/Tuner/tensorboards/14/iris_tune_model
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
dnn_hidden_layer_0: 135
dnn_hidden_layer_1: 63
Score: 0.9200000166893005
Trial summary
Hyperparameters:
dnn_hidden_layer_0: 120
dnn_hidden_layer_1: 51
Score: 0.8999999761581421
Trial summary
Hyperparameters:
dnn_hidden_layer_0: 107
dnn_hidden_layer_1: 60
Score: 0.8899999856948853
Trial summary
Hyperparameters:
dnn_hidden_layer_0: 128
dnn_hidden_layer_1: 50
Score: 0.8899999856948853
Trial summary
Hyperparameters:
dnn_hidden_layer_0: 105
dnn_hidden_layer_1: 55
Score: 0.8899999856948853
Trial summary
Hyperparameters:
dnn_hidden_layer_0: 133
dnn_hidden_layer_1: 55
Score: 0.8600000143051147
Processing ./pipelines/iris/_wheels/tfx_user_code_Trainer-0.0+cdd0b8bea05713bd18c34d96752b4a5d9ef743c4bb5d652b4b7b2982c29698a3-py3-none-any.whl


INFO:absl:Successfully installed 'pipelines/iris/_wheels/tfx_user_code_Trainer-0.0+cdd0b8bea05713bd18c34d96752b4a5d9ef743c4bb5d652b4b7b2982c29698a3-py3-none-any.whl'.
INFO:absl:Training model.
INFO:absl:Feature class has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature petallength has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature petalwidth has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature sepallength has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature sepalwidth has a shape dim {
  size: 1
}
. Setting to DenseTensor.


...

2021-11-24 00:09:48.525798: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-11-24 00:09:48.525825: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.


  41/2000 [..............................] - ETA: 23s - loss: 0.1797 - accuracy: 0.9268

2021-11-24 00:09:48.797879: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-11-24 00:09:48.798946: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1748] CUPTI activity buffer flushed
2021-11-24 00:09:48.820897: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 89 callback api events and 86 activity events. 
2021-11-24 00:09:48.823248: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-11-24 00:09:48.826328: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: pipelines/iris/Trainer/model_run/15/train/plugins/profile/2021_11_24_00_09_48

2021-11-24 00:09:48.828363: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to pipelines/iris/Trainer/model_run/15/train/plugins/profile/2021_11_24_00_09_48/EkE-Legion5.trace.json.gz
2021-11-24 00:09:48.832607: I tensorflow/core/profiler/rpc/clie

2000/2000 [==============================] - 9s 5ms/step - loss: 0.0055 - accuracy: 0.9978 - val_loss: 1.5922 - val_accuracy: 0.9000
INFO:tensorflow:Assets written to: pipelines/iris/Trainer/model/15/Format-Serving/assets


INFO:tensorflow:Assets written to: pipelines/iris/Trainer/model/15/Format-Serving/assets
INFO:absl:Training complete. Model written to pipelines/iris/Trainer/model/15/Format-Serving. ModelRun written to pipelines/iris/Trainer/model_run/15
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 15 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'model_run': [Artifact(artifact: uri: "pipelines/iris/Trainer/model_run/15"
custom_properties {
  key: "name"
  value {
    string_value: "iris:2021-11-24T00:09:20.906442:Trainer:model_run:0"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.4.0"
  }
}
, artifact_type: name: "ModelRun"
)], 'model': [Artifact(artifact: uri: "pipelines/iris/Trainer/model/15"
custom_properties {
  key: "name"
  value {
    string_value: "iris:2021-11-24T00:09:20.906442:Trainer:model:0"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    strin

# Visualize pipeline

In [8]:
from tfx.orchestration.portable.mlmd import execution_lib
from tfx.orchestration.metadata import Metadata
from tfx.orchestration.experimental.interactive import visualizations
from ml_metadata.proto import metadata_store_pb2

def get_latest_artifact(component_id):
    metadata_connection_config = tfx.orchestration.metadata.sqlite_metadata_connection_config(METADATA_PATH)

    with Metadata(metadata_connection_config) as metadata_handler:
        context = metadata_handler.store.get_context_by_type_and_name('node', f'{PIPELINE_NAME}.{component_id}')
        executions = metadata_handler.store.get_executions_by_context(context.id)
        latest_execution = max(executions, key=lambda e: e.last_update_time_since_epoch)

        return execution_lib.get_artifacts_dict(metadata_handler, latest_execution.id, [metadata_store_pb2.Event.OUTPUT])


def visualize(artifact):
    visualization = visualizations.get_registry().get_visualization(artifact.type_name)
    visualization.display(artifact)


from tfx.orchestration.experimental.interactive import standard_visualizations

standard_visualizations.register_standard_visualizations()

%load_ext tensorboard

## StatisticsGen

In [9]:
statistic_artifact = get_latest_artifact("StatisticsGen")['statistics'][0]

visualize(statistic_artifact)

INFO:absl:MetadataStore with DB connection initialized


## SchemaGen

In [10]:
schema_artifact = get_latest_artifact("SchemaGen")['schema'][0]

visualize(schema_artifact)

INFO:absl:MetadataStore with DB connection initialized


,Type,Presence,Valency,Domain
Feature name,,,,
'class',STRING,required,,'class'
'petallength',FLOAT,required,,-
'petalwidth',FLOAT,required,,-
'sepallength',FLOAT,required,,-
'sepalwidth',FLOAT,required,,-


,Values
Domain,
'class',"'Iris-setosa', 'Iris-versicolor', 'Iris-virginica'"


## ExampleValidator

In [11]:
example_validator_articaft = get_latest_artifact("ExampleValidator")['anomalies'][0]

visualize(example_validator_articaft)

INFO:absl:MetadataStore with DB connection initialized


## Tuner

In [12]:
tuner_articaft = get_latest_artifact("Tuner")["best_hyperparameters"][0]

tuner_tensorboard = os.path.join(
    tuner_articaft.uri[0:(len('Tuner') + tuner_articaft.uri.index('Tuner'))],
    "tensorboards",
    os.path.split(tuner_articaft.uri.strip("/"))[-1]
)


%tensorboard --logdir {tuner_tensorboard}

INFO:absl:MetadataStore with DB connection initialized


## Trainer

In [13]:
model_run_artifact_dir = get_latest_artifact("Trainer")['model_run'][0].uri

%tensorboard --logdir {model_run_artifact_dir}

INFO:absl:MetadataStore with DB connection initialized


# Prediction

Serve model by running this in your shell:

```
$ tensorflow_model_server --rest_api_port=8501 --model_name=iris --model_base_path=.../tfx_seminar/serving_model/iris
```

In [14]:
import json
import requests

URL = 'http://localhost:8501/v1/models/' + PIPELINE_NAME + ':predict'
HEADERS = {"content-type": "application/json"}

def prediction_request(sepallength, sepalwidth, petallength, petalwidth):
    request_data = json.dumps({"instances": [{
        "petallength": [petallength],
        "petalwidth": [petalwidth],
        "sepallength": [sepallength],
        "sepalwidth": [sepalwidth]
        }]
    })
    
    response = requests.post(data=request_data, url=URL, headers=HEADERS)
    
    return json.loads(response.text)

prediction_request(4.4,3.2,1.3,0.2)

ConnectionError: HTTPConnectionPool(host='localhost', port=8501): Max retries exceeded with url: /v1/models/iris:predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f9a1451fa00>: Failed to establish a new connection: [Errno 111] Connection refused'))